In [2]:
import numpy as np
import pandas as pd
import io
import sys

from io import StringIO

for p in (np, pd):
    print(p.__name__, p.__version__)

numpy 1.18.1
pandas 1.0.1


> **LC 1149**

Medium

Linkedin phone interview question

There is no primary key for this table, it may have duplicate rows.
Each row of this table indicates that some viewer viewed an article (written by some author) on some date. 
Note that equal author_id and viewer_id indicate the same person.


In [35]:
data = StringIO(
"""
article_id,author_id,viewer_id,view_date
1,3,5,2019-08-01 
3,4,5,2019-08-01 
1,3,6,2019-08-02 
2,7,7,2019-08-01 
2,7,6,2019-08-02 
4,7,1,2019-07-22 
3,4,4,2019-07-21 
3,4,4,2019-07-21 

"""
)

 
df = (
    pd.read_csv(data, sep=",")
      .assign(
    view_date=lambda x: pd.to_datetime(x['view_date'])
)
)

df.head(20)

,article_id,author_id,viewer_id,view_date
0,1,3,5,2019-08-01
1,3,4,5,2019-08-01
2,1,3,6,2019-08-02
3,2,7,7,2019-08-01
4,2,7,6,2019-08-02
5,4,7,1,2019-07-22
6,3,4,4,2019-07-21
7,3,4,4,2019-07-21


Q1. find all the people who viewed more than one article on the same date, sorted in ascending order by their id.

The solution in SQL 

```SQL

    SELECT viewer_id
    FROM views
    GROUP BY viewer_id, view_date
    HAVING COUNT(DISTINCT article_id) > 1
    ORDER BY viewer_id

```

`HAVING` is translated to `groupby` followed by `agg` and `loc`


In [44]:
(
    df.groupby(['viewer_id', 'view_date'], as_index=False)
    .agg({'article_id': pd.Series.nunique})
    .loc[lambda x: x.article_id > 1, ['viewer_id']]
)

,viewer_id
2,5
3,6


Q2. How many members viewed more than one article on 2019-08-01?

```SQL

SELECT viewer_id
FROM Views
WHERE CONVERT(DATE, view_date) = CAST('2019-08-01' AS DATE) 
GROUP BY viewer_id
HAVING COUNT(DISTINCT article_id) > 1


```


In [48]:
(
    df
    .loc[df.view_date == '2019-08-01']
    .groupby(['viewer_id', 'view_date'], as_index=False)
    .agg({'article_id': pd.Series.nunique})
    .loc[lambda x: x.article_id > 1, ['viewer_id']]
)

,viewer_id
0,5


Q3. How many article authors have never viewed their own article?

```SQL
    SELECT COUNT(author_id)
    FROM Views
    WHERE author_id NOT IN (
    SELECT DISTINCT author_id 
    FROM Views
    WHERE author_id == viewer_id
)

```


In [49]:
(
    df
    .loc[~df.author_id
         .isin(df.loc[df.author_id == df.viewer_id, 'author_id']
               .unique())
        ]
)


,article_id,author_id,viewer_id,view_date
0,1,3,5,2019-08-01
2,1,3,6,2019-08-02


> **LC 615**



In [53]:
salary_data = StringIO(
"""
id,employee_id,amount,pay_date
1 ,1          ,9000  ,2017-03-31
2 ,2          ,6000  ,2017-03-31
3 ,3          ,10000 ,2017-03-31
4 ,1          ,7000  ,2017-02-28
5 ,2          ,6000  ,2017-02-28
6 ,3          ,8000  ,2017-02-28

"""
)

 
df_salary = (
    pd.read_csv(salary_data, sep=",")
      .assign(
    pay_date=lambda x: pd.to_datetime(x['pay_date'])
)
)

df_salary.head(20)

,id,employee_id,amount,pay_date
0,1,1,9000,2017-03-31
1,2,2,6000,2017-03-31
2,3,3,10000,2017-03-31
3,4,1,7000,2017-02-28
4,5,2,6000,2017-02-28
5,6,3,8000,2017-02-28


In [54]:
employee_data = StringIO(
"""

employee_id,department_id
1          ,1            
2          ,2            
3          ,2            

"""

)

df_employee = (
    pd.read_csv(employee_data, sep=",")
)

df_employee.head(20)

,employee_id,department_id
0,1,1
1,2,2
2,3,2


Q1. Given two tables as below, write a query to display the comparison result (higher/lower/same) of the average salary of employees in a department to the company’s average salary.

```SQL

    WITH dept_avg_salary AS (
      SELECT department_id, LEFT(CONVERT(VARCHAR, pay_date, 23), 7) AS month, AVG(Amount) as dept_avg_salary
      FROM Salary AS s JOIN Employee AS e ON (
            s.employee_id = e.employee_id
      )
      GROUP BY department_id, LEFT(CONVERT(VARCHAR, pay_date, 23), 7)
    ), company_avg_salary AS (
      SELECT LEFT(CONVERT(VARCHAR, pay_date, 23), 7) AS month, AVG(Amount) AS company_avg_salary
      FROM Salary
      GROUP BY LEFT(CONVERT(VARCHAR, pay_date, 23), 7)
    )
    SELECT department_id, d.month,
        (
            CASE  
                WHEN dept_avg_salary < company_avg_salary THEN 'LOWER'
                WHEN dept_avg_salary > company_avg_salary  THEN 'HIGHER'
                ELSE 'SAME'
            END
        ) AS dept_vs_company
    FROM dept_avg_salary AS d JOIN company_avg_salary AS c ON (
            d.month = c.month
 )

```

In [63]:
# how to groupby year and month, pipe temp into groupby and reference columns?
# as_index = False will NOT output the two groupby columns
df_dept = (pd.merge(left=df_salary, right=df_employee, left_on='employee_id', right_on='employee_id'))
df_dept = df_dept.groupby([df_dept.department_id, df_dept.pay_date.dt.strftime("%Y-%m")])['amount'].mean().reset_index()
df_dept

,department_id,pay_date,amount
0,1,2017-02,7000
1,1,2017-03,9000
2,2,2017-02,7000
3,2,2017-03,8000


In [71]:
df = (
    df_salary
    .pipe(pd.merge, right=df_employee, left_on='employee_id', right_on='employee_id')
    .assign(dept_avg = lambda x: x.groupby([x.department_id, x.pay_date.dt.strftime("%Y-%m")])['amount'].transform('mean'))
    .assign(company_avg = lambda x: x.groupby(x.pay_date.dt.strftime("%Y-%m")).amount.transform('mean'))
)
df

,id,employee_id,amount,pay_date,department_id,dept_avg,company_avg
0,1,1,9000,2017-03-31,1,9000,8333.333333
1,4,1,7000,2017-02-28,1,7000,7000.000000
2,2,2,6000,2017-03-31,2,8000,8333.333333
3,5,2,6000,2017-02-28,2,7000,7000.000000
4,3,3,10000,2017-03-31,2,8000,8333.333333
5,6,3,8000,2017-02-28,2,7000,7000.000000


> **LinkedIn Question 3**

In [3]:
df = pd.DataFrame(
    {'Member_ID': [1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4],
     'Company': ['Microsoft', 'Google', 'Facebook',
                 'Microsoft', 'Google', 'Oracle',
                 'Microsoft', 'Oracle', 'Microsoft', 'Google',
                 'Microsoft', 'Apple', 'Google'
                ],
     'Year_Start': [2000, 2006, 2012, 
                    2001, 2007, 2004,
                    2001, 2002, 2003, 2004,
                    2004, 2005, 2006
                   ]
   }
)

df

,Member_ID,Company,Year_Start
0,1,Microsoft,2000
1,1,Google,2006
2,1,Facebook,2012
3,2,Microsoft,2001
4,2,Google,2007
5,2,Oracle,2004
6,3,Microsoft,2001
7,3,Oracle,2002
8,3,Microsoft,2003
9,3,Google,2004


Q3: How many people went to Google from Microsoft directly and indirectly?

ANS: 4

```SQL

    SELECT COUNT(DISTINCT e1.Member_ID) AS Number_Employees
    FROM emp_history AS e1 JOIN emp_history AS e2 ON (
        
        e1.Member_ID == e2.Member_ID
    )
    WHERE e1.Company == 'Microsoft' AND e2.company == 'Google' AND (
        e1.Year_Start < e2.Year_Start
    )

```

In [76]:
df.loc[df.Company.isin(('Microsoft', 'Google'))].pipe(
    pd.merge, right=df.loc[df.Company.isin(('Microsoft', 'Google'))], 
    left_on='Member_ID', right_on='Member_ID', suffixes=('_1', '_2')
).loc[lambda x: (x.Company_1 == 'Microsoft') & (x.Company_2 == 'Google') &
     (x.Year_Start_1 < x.Year_Start_2)]

,Member_ID,Company_1,Year_Start_1,Company_2,Year_Start_2
1,1,Microsoft,2000,Google,2006
5,2,Microsoft,2001,Google,2007
10,3,Microsoft,2001,Google,2004
13,3,Microsoft,2003,Google,2004


Q4: How many people went to Google from Microsoft directly?

ANS = 2

```SQL
    
    WITH emp_history_index AS (
        SELECT Member_ID, Company, Year_Start, 
        ROW_NUMBER() OVER(PARTITION BY Member_ID ORDER BY Year_Start) AS Seq
        FROM Emp_History
    )
    SELECT e1.Member_ID
    FROM emp_history_index AS e1 JOIN emp_history_index AS e2 ON (
    
        e1.Member_ID = e2.Member_ID AND
        e2.Seq - e1.Seq = 1
    )
    WHERE e1.Company LIKE 'Microsoft' and e2.Company LIKE 'Google'
```


In [87]:
df

,Year_Start
0,1
1,2
2,3
3,1
4,3
5,2
6,1
7,2
8,3
9,4


In [94]:
df_temp = (df
 .assign(seq=lambda x: 
         x.groupby('Member_ID',as_index=False)['Year_Start']
         .rank().astype('int'))
 .loc[lambda x: x.Company.isin(('Google','Microsoft'))]
          )

df_temp.pipe(pd.merge, 
             right=df_temp, 
             left_on='Member_ID', right_on='Member_ID',
             suffixes=('_1', '_2')).loc[lambda x: (x.Company_1=='Microsoft') &
    (x.Company_2 == 'Google') & (x.seq_2 - x.seq_1 == 1)
    ]


,Member_ID,Company_1,Year_Start_1,seq_1,Company_2,Year_Start_2,seq_2
1,1,Microsoft,2000,1,Google,2006,2
13,3,Microsoft,2003,3,Google,2004,4


Q5: How many people went from Microsoft to Google and stayed there?

```SQL

    WITH last_year AS (
        SELECT Member_ID, 
        MAX(Year_Start) last_year
        FROM Emp_History 
        GROUP BY Member_ID
    ), last_job AS (
        SELECT e.Member_ID, e.Company, Last_Year
        FROM emp_history AS e JOIN last_year AS l ON (
            e.Member_ID = l.Member_ID AND
            e.Year_Start = l.Last_Year
        )
    
    )
    SELECT DISTINCT (e.Member_ID)
    FROM emp_history AS e JOIN last_job AS l ON (
        
        e.Member_ID = l.Member_ID AND
        e.Year_Start < l.last_year
    )
    WHERE e.Company LIKE 'Microsoft' AND l.company LIKE 'Google'
    


```

In [9]:
df.groupby('Member_ID', as_index=False)[['Company', 'Year_Start']].last()

,Member_ID,Company,Year_Start
0,1,Facebook,2012
1,2,Oracle,2004
2,3,Google,2004
3,4,Google,2006


> **Linkedin Question 4**



In [98]:
# Assume the status table represents status as of 2019-07-01
status = pd.DataFrame(
    {'member_id': [1, 2, 3, 4],
     'status': ['on', 'off', 'on', 'off'],
   }
)

# Members can turn on/off the status
actions = pd.DataFrame(
    {'member_id': [1, 1, 2, 4, 4, 4, 5],
     'date_sk': ['2019-07-02',
                 '2019-07-05',
                 '2019-07-03',
                 '2019-07-10',
                 '2019-07-13',
                 '2019-07-16',
                 '2019-07-09',
                ],
     'action': ['turn_off',
                'turn_on',
                'turn_on',
                'turn_on',
                'turn_on',
                'turn_off',
                'turn_on',         
               ],
    }
)

In [99]:
status

,member_id,status
0,1,on
1,2,off
2,3,on
3,4,off


In [100]:
actions

,member_id,date_sk,action
0,1,2019-07-02,turn_off
1,1,2019-07-05,turn_on
2,2,2019-07-03,turn_on
3,4,2019-07-10,turn_on
4,4,2019-07-13,turn_on
5,4,2019-07-16,turn_off
6,5,2019-07-09,turn_on


Question: Latest status for each member?

```SQL

WITH action_seq AS (
    SELECT member_id, action, 
    ROW_NUMBER() OVER (
        PARTITION BY member_id 
        ORDER BY date_sk DESC
    ) AS seq
    FROM actions
), last_action AS (
    SELECT member_id, action AS last_action
    FROM action_seq
    WHERE seq = 1   
), status_action AS (
SELECT s.member_id AS member_id1, 
       l.member_id AS member_id2, status, 
    (
        CASE 
            WHEN last_action LIKE 'turn_off' THEN 'off'
            WHEN last_action LIKE 'turn_on' THEN 'on'
        END
    ) AS last_action
FROM status AS s FULL OUTER JOIN last_action AS l ON (
          s.member_id = l.member_id
    )
)
SELECT COALESCE(member_id1, member_id2) AS member_id, 
(
  CASE 
    WHEN last_action IS NOT NULL THEN last_action
    WHEN last_action is NULL then status
  END
) AS status
FROM status_action
    

```


In [103]:
df_status = status.assign(date_sk = '2019-07-01')
df_action = actions.rename(columns={'action':'status'}).assign(status=lambda x: np.where(x.status=='turn_on', 'on', 'off'))
df = pd.concat([df_status, df_action]).sort_values(by=['member_id', 'date_sk'])
df.groupby('member_id').status.last()

member_id
1     on
2     on
3     on
4    off
5     on
Name: status, dtype: object